# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [1]:
import matplotlib.pyplot as plt
import PIL.Image as PIL_image
import numpy as np

from importlib.machinery import SourceFileLoader
#In case of importing files from other directory
clip_pb2 = SourceFileLoader("clip_pb2","../protos/clip_pb2.py").load_module()
clip_pb2_grpc = SourceFileLoader("clip_pb2_grpc","../protos/clip_pb2_grpc.py").load_module()

import grpc
import io

## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data and call the service

In [32]:
from PIL import Image
import torch

#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'

texts = ["a diagram", "a dog", "a cat", "garden", "grass", "dirt road"]

file_paths = ['dog.jpg']

image_byte_list = []
for path in file_paths:
    # Open the file in binary read mode ('rb') and read its entire content
    with open(path, 'rb') as f:
        image_bytes = f.read()
        image_byte_list.append(image_bytes)
        print(f"Read {path}: {len(image_bytes) / (1024 * 1024):.2f} MB")




request = clip_pb2.CLIPRequest(images= image_byte_list,
                               texts=texts) # list of strings

# List of file paths (example paths — replace with your actual ones)

channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = clip_pb2_grpc.CLIPServiceStub(channel)

Read dog.jpg: 0.37 MB


In [33]:
#Name of the method in the service
response = estimator_stub.Forward(request)
#After finishing channel.close


In [40]:
torch.load(io.BytesIO(response.similarity)).softmax(dim=-1)

tensor([[1.4569e-03, 3.1287e-01, 5.5226e-03, 8.3366e-03, 6.7161e-01, 1.9901e-04]])

In [43]:
torch.load(io.BytesIO(response.text_emb)).shape

torch.Size([6, 512])